In [2]:
import polars as pl 

# Explore Metadata

In [24]:
metadata_df = pl.read_csv("../data/adex-database/metadata.tsv", separator="\t")

In [25]:
metadata_df

Sample,GSE,Experimental Strategy,GPL,Condition,Tissue,Cell Type,Gender,Age,Ethnicity
str,str,str,str,str,str,str,str,str,str
"""GSM260887""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260890""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260892""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260895""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260898""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260901""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260904""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260907""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""
"""GSM260910""","""GSE10325""","""Expression""","""GPL96""","""Healthy""","""Peripheral blo…","""B cells""","""NA""","""NA""","""NA"""


In [29]:
total_samples = metadata_df.select("Sample").shape[0]
unique_samples = metadata_df.select("Sample").unique().shape[0]

conditions_df = metadata_df.select("Condition").unique()
conditions_count = conditions_df.shape[0]
conditions = conditions_df.to_series().to_list()

tissues_df = metadata_df.select("Tissue").unique()
tissues_count = tissues_df.shape[0]
tissues = tissues_df.to_series().to_list()

gpl_df = metadata_df.select("GPL").unique()
gpl_count = gpl_df.shape[0]
gpls = gpl_df.to_series().to_list()

males = metadata_df.select("Gender").filter(pl.col("Gender") == "Male").shape[0]
females = metadata_df.select("Gender").filter(pl.col("Gender") == "Female").shape[0]
gender_unknown = metadata_df.select("Gender").filter((pl.col("Gender") != "Female") & (pl.col("Gender") != "Male")).shape[0]

print(f"""
Total Samples = {total_samples}
Unique Samples = {unique_samples}
Conditions = {conditions_count} [{",".join(conditions)}]
GPL = {gpl_count} [{",".join(gpls)}]
Tissues = {tissues_count} [{",".join(tissues)}]
Males = {males}, Females = {females}, Unknown Gender = {gender_unknown}
""")


Total Samples = 3980
Unique Samples = 3854
Conditions = 6 [SjS,SSc,SLE,T1D,Healthy,RA]
GPL = 13 [GPL13158,GPL10558,GPL570,GPL96,GPL97,GPL15456,GPL10558_GPL13534,GPL20301,GPL6884,GPL11154,GPL16791,GPL14951_GPL13534,GPL13667]
Tissues = 7 [Parotid gland,Salivary gland,Saliva,Whole blood,Synovial membrane,Skin,Peripheral blood]
Males = 269, Females = 1547, Unknown Gender = 2164


In [27]:
metadata_df.group_by("Condition").agg(pl.struct(["Sample"]).n_unique().alias('SamplesPerCondition')).sort("SamplesPerCondition", descending=True)

Condition,SamplesPerCondition
str,u32
"""SLE""",1502
"""Healthy""",957
"""RA""",862
"""SjS""",281
"""SSc""",157
"""T1D""",95


In [28]:
metadata_df.group_by("Age").agg(pl.struct(["Sample"]).n_unique().alias('SamplesPerAge')).sort("Age")


Age,SamplesPerAge
str,u32
"""1-10""",113
"""11-20""",342
"""21-30""",383
"""31-40""",331
"""41-50""",105
"""51-60""",66
"""61-70""",69
"""71-80""",39
""">80""",6


# Explore Files that seem problematic

### GSE95065

There seem to be two files that need to be merged into one 


In [3]:
GSE95065_large = pl.read_parquet("../data/adex-database/samples/SSc/GSE95065.tsv.parquet")
GSE95065_large

gene,GSM2495594,GSM2495595,GSM2495596,GSM2495597,GSM2495598,GSM2495599,GSM2495600,GSM2495601,GSM2495602,GSM2495603,GSM2495604,GSM2495605,GSM2495606,GSM2495607,GSM2495608,GSM2495609,GSM2495610,GSM2495611,GSM2495612,GSM2495613,GSM2495614,GSM2495615,GSM2495616,GSM2495617,GSM2495618,GSM2495619,GSM2495620,GSM2495621,GSM2495622,GSM2495623,GSM2495624,GSM2495625,GSM2495626
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""RFC2""",7.472319,7.520892,7.46077,7.708487,7.484799,7.299081,7.403904,7.419463,7.45063,7.61232,7.46854,7.532882,7.411745,7.435213,7.392205,7.532592,7.403685,7.449712,7.52772,7.240532,7.633632,7.264277,7.582619,7.21455,7.082178,7.386463,7.320884,7.227947,7.456815,7.379748,7.278312,7.269157,7.1024
"""HSPA6""",6.655682,6.248444,6.563427,6.74514,6.030863,5.127361,6.477777,5.914106,6.815282,6.247377,6.455072,6.769109,5.906113,6.368727,6.104465,6.300024,6.50896,6.381587,6.293698,6.213825,6.221177,7.188617,5.930373,6.517476,5.987333,6.650793,6.977275,6.185656,7.468024,6.119981,6.749708,6.33245,6.380119
"""PAX8""",6.772054,6.926438,7.034588,6.975327,7.220449,6.916914,6.659487,6.965389,6.622051,7.281831,7.033667,6.778211,7.172063,7.045818,7.006107,6.899453,7.061638,7.335806,6.919635,6.931886,6.94088,6.986259,7.180362,6.525082,7.144469,6.556937,6.600135,7.149994,6.662156,7.145639,7.119249,7.113005,6.452059
"""THRA""",6.90694,6.48762,6.229516,6.134647,6.217019,6.197873,5.779039,6.464217,6.489212,6.001884,6.219632,6.309929,5.870903,5.843094,6.005228,6.168541,5.932801,6.147186,5.98024,5.860218,5.608119,5.963343,6.404039,6.674311,6.161022,6.13027,6.000803,5.985383,5.773169,5.903811,6.093964,5.95448,6.172006
"""PTPN21""",6.99916,7.081638,7.270245,6.609682,7.529765,6.601726,6.198417,7.849579,6.695222,6.546318,6.706809,7.358162,7.494944,7.113105,6.591536,7.221406,6.52857,6.701473,7.611369,7.117925,6.774631,6.386196,7.258027,6.129345,6.430386,6.517319,6.941436,6.722773,6.443518,7.459997,7.135372,7.249807,7.095118
"""CCL5""",7.07975,6.092999,5.735654,6.728099,5.811003,4.609014,8.603649,6.084578,7.141292,5.4493,5.653901,7.069109,5.446654,6.361946,4.8171,5.031157,4.938001,5.160383,6.333225,4.035719,4.729835,6.90157,5.817311,6.325495,5.580433,5.583964,8.38015,6.218533,6.087392,8.100331,5.367073,5.649418,5.039789
"""CYP2E1""",6.196167,4.492376,6.459971,6.627319,6.291382,5.109184,5.991173,4.886798,5.518165,5.869055,6.285918,6.374689,6.242608,7.109768,6.597216,4.919737,6.845075,5.972443,6.209479,6.62767,5.960389,4.929532,5.697879,5.336928,6.289446,4.41045,6.09204,5.686044,5.875945,5.935592,6.07799,6.337083,5.81603
"""EPHB3""",6.958701,6.994674,7.523542,7.152963,7.115558,7.486647,6.428781,7.50271,6.905031,7.73239,7.597305,7.363381,7.893089,7.599454,8.193947,7.752947,7.931709,7.831751,7.553464,7.979561,7.743131,7.468185,7.82446,7.405587,7.720182,7.507602,7.167658,7.26325,7.153739,7.480215,7.481708,7.64475,7.53078
"""ESRRA""",6.80847,6.797286,6.954059,7.001706,6.857725,7.050968,6.814291,6.938178,6.786193,7.206581,7.022166,6.916465,7.041363,7.029736,7.339206,6.988665,7.099916,7.382145,6.832798,7.048717,7.14127,6.830311,7.200086,6.789154,7.324337,6.704587,6.626802,6.917285,6.828058,6.940019,6.931476,7.230816,7.018688


In [5]:
GSE95065_small = pl.read_parquet("../data/adex-database/samples/SSc/GSE95065_2.tsv.parquet")
GSE95065_small

gene,GSM2495594,GSM2495595,GSM2495596,GSM2495597,GSM2495598,GSM2495599,GSM2495600,GSM2495601,GSM2495602,GSM2495603,GSM2495604,GSM2495605,GSM2495606,GSM2495607,GSM2495608,GSM2495609,GSM2495610,GSM2495611,GSM2495612,GSM2495613,GSM2495614,GSM2495615,GSM2495616,GSM2495617,GSM2495618,GSM2495619,GSM2495620,GSM2495621,GSM2495622,GSM2495623,GSM2495624,GSM2495625,GSM2495626
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""CEBPE""",7.430844,7.424512,7.380993,7.495515,7.424263,7.24761,7.358499,7.352183,7.373819,7.574839,7.341107,7.483675,7.384654,7.317921,7.326927,7.497311,7.343509,7.384384,7.439529,7.171636,7.541954,7.175672,7.52662,7.103011,7.094538,7.276562,7.277203,7.154872,7.350882,7.365737,7.076975,7.140006,7.11732
"""ADCYAP1R1""",5.94925,5.912904,5.952583,6.136321,5.660838,4.507445,5.995402,5.392767,6.156154,5.868257,5.963205,6.043992,5.441751,5.91522,5.651909,5.814725,6.128093,5.859722,5.756369,5.602133,5.668191,6.470656,5.523302,5.818856,5.381958,5.952577,6.267259,5.528522,6.896724,5.5735,6.109171,5.765169,5.834804
"""COL7A1""",6.533028,6.441243,6.26865,6.402277,6.406926,6.124735,6.913235,6.183706,6.143744,6.092093,6.323905,6.448042,6.687022,6.449526,6.58893,6.058564,6.496673,6.376913,6.206048,6.574422,6.464814,6.64254,6.144032,6.473642,6.567916,6.610596,6.442491,6.794229,6.435983,6.277183,6.302032,6.382896,6.893414
"""KLF6""",6.90694,6.48762,6.229516,6.134647,6.217019,6.197873,5.779039,6.464217,6.489212,6.001884,6.226221,6.309929,5.870903,5.843094,5.924252,6.168541,6.366553,6.147186,5.792084,6.085894,5.959874,5.963343,6.404039,6.674311,6.161022,6.13027,6.000803,5.985383,5.773169,5.903811,6.093964,5.95448,6.172006
"""CSF2RA""",5.482237,5.547813,6.167729,5.801479,5.7241,6.074025,5.030489,5.985279,5.512681,7.077504,6.963825,5.931991,7.247194,7.027666,7.610069,6.462201,7.18134,7.206354,6.125501,7.399473,7.054716,6.050846,7.134491,6.054555,7.198266,6.117478,5.848947,5.850248,5.67489,5.840013,6.061546,6.261048,6.09999
"""CTBP1""",7.096109,7.070467,7.344943,7.401909,7.352289,7.293874,6.940814,7.268688,7.075635,7.617213,7.378066,7.332296,7.393262,7.464938,7.756849,7.379056,7.383231,7.848183,7.146071,7.513861,7.4345,7.320879,7.648898,7.124524,7.703255,7.059668,6.957033,7.352952,7.183082,7.296272,7.286571,7.572689,7.418754
"""DIAPH1""",8.14523,8.312656,7.992397,8.272858,8.040403,7.828828,8.157008,8.061688,8.004696,7.385955,7.887343,8.037436,8.048556,7.902219,7.907266,8.116818,7.760369,7.803573,8.014856,7.750942,7.780054,8.198553,8.027947,7.939627,7.718404,8.141551,8.181171,8.108116,8.095633,7.884621,7.977461,7.938129,8.151805
"""AGER""",6.496517,6.508898,6.538657,6.435531,6.282903,5.896218,6.146781,6.536278,6.593639,6.228704,6.463922,6.752157,6.412501,6.361711,6.306303,6.793278,6.331338,6.189678,6.208045,6.186679,6.070997,6.663842,6.381444,6.595394,5.902664,6.686002,6.480337,5.997034,6.535427,6.661929,6.405839,6.227995,6.342333
"""DNASE1""",4.872944,4.847915,5.046468,5.162639,5.052784,4.968124,4.846196,4.997902,4.970702,6.169151,6.330242,4.943527,6.44802,6.408799,6.298719,5.029668,6.074697,6.764309,4.955496,6.080928,6.642391,4.991812,6.655077,4.973225,6.002208,4.765483,4.947005,4.872625,5.144602,5.031794,5.089739,4.942311,5.133462


In [ ]:
# Make certain that we have the same samples in the two files 
large_set_samples = set(GSE95065_large.columns)
small_set_samples = set(GSE95065_small.columns)
large_set_samples == small_set_samples

In [13]:
GSE95065_merged = pl.concat([GSE95065_small, GSE95065_large], rechunk=True)
GSE95065_merged

gene,GSM2495594,GSM2495595,GSM2495596,GSM2495597,GSM2495598,GSM2495599,GSM2495600,GSM2495601,GSM2495602,GSM2495603,GSM2495604,GSM2495605,GSM2495606,GSM2495607,GSM2495608,GSM2495609,GSM2495610,GSM2495611,GSM2495612,GSM2495613,GSM2495614,GSM2495615,GSM2495616,GSM2495617,GSM2495618,GSM2495619,GSM2495620,GSM2495621,GSM2495622,GSM2495623,GSM2495624,GSM2495625,GSM2495626
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""CEBPE""",7.430844,7.424512,7.380993,7.495515,7.424263,7.24761,7.358499,7.352183,7.373819,7.574839,7.341107,7.483675,7.384654,7.317921,7.326927,7.497311,7.343509,7.384384,7.439529,7.171636,7.541954,7.175672,7.52662,7.103011,7.094538,7.276562,7.277203,7.154872,7.350882,7.365737,7.076975,7.140006,7.11732
"""ADCYAP1R1""",5.94925,5.912904,5.952583,6.136321,5.660838,4.507445,5.995402,5.392767,6.156154,5.868257,5.963205,6.043992,5.441751,5.91522,5.651909,5.814725,6.128093,5.859722,5.756369,5.602133,5.668191,6.470656,5.523302,5.818856,5.381958,5.952577,6.267259,5.528522,6.896724,5.5735,6.109171,5.765169,5.834804
"""COL7A1""",6.533028,6.441243,6.26865,6.402277,6.406926,6.124735,6.913235,6.183706,6.143744,6.092093,6.323905,6.448042,6.687022,6.449526,6.58893,6.058564,6.496673,6.376913,6.206048,6.574422,6.464814,6.64254,6.144032,6.473642,6.567916,6.610596,6.442491,6.794229,6.435983,6.277183,6.302032,6.382896,6.893414
"""KLF6""",6.90694,6.48762,6.229516,6.134647,6.217019,6.197873,5.779039,6.464217,6.489212,6.001884,6.226221,6.309929,5.870903,5.843094,5.924252,6.168541,6.366553,6.147186,5.792084,6.085894,5.959874,5.963343,6.404039,6.674311,6.161022,6.13027,6.000803,5.985383,5.773169,5.903811,6.093964,5.95448,6.172006
"""CSF2RA""",5.482237,5.547813,6.167729,5.801479,5.7241,6.074025,5.030489,5.985279,5.512681,7.077504,6.963825,5.931991,7.247194,7.027666,7.610069,6.462201,7.18134,7.206354,6.125501,7.399473,7.054716,6.050846,7.134491,6.054555,7.198266,6.117478,5.848947,5.850248,5.67489,5.840013,6.061546,6.261048,6.09999
"""CTBP1""",7.096109,7.070467,7.344943,7.401909,7.352289,7.293874,6.940814,7.268688,7.075635,7.617213,7.378066,7.332296,7.393262,7.464938,7.756849,7.379056,7.383231,7.848183,7.146071,7.513861,7.4345,7.320879,7.648898,7.124524,7.703255,7.059668,6.957033,7.352952,7.183082,7.296272,7.286571,7.572689,7.418754
"""DIAPH1""",8.14523,8.312656,7.992397,8.272858,8.040403,7.828828,8.157008,8.061688,8.004696,7.385955,7.887343,8.037436,8.048556,7.902219,7.907266,8.116818,7.760369,7.803573,8.014856,7.750942,7.780054,8.198553,8.027947,7.939627,7.718404,8.141551,8.181171,8.108116,8.095633,7.884621,7.977461,7.938129,8.151805
"""AGER""",6.496517,6.508898,6.538657,6.435531,6.282903,5.896218,6.146781,6.536278,6.593639,6.228704,6.463922,6.752157,6.412501,6.361711,6.306303,6.793278,6.331338,6.189678,6.208045,6.186679,6.070997,6.663842,6.381444,6.595394,5.902664,6.686002,6.480337,5.997034,6.535427,6.661929,6.405839,6.227995,6.342333
"""DNASE1""",4.872944,4.847915,5.046468,5.162639,5.052784,4.968124,4.846196,4.997902,4.970702,6.169151,6.330242,4.943527,6.44802,6.408799,6.298719,5.029668,6.074697,6.764309,4.955496,6.080928,6.642391,4.991812,6.655077,4.973225,6.002208,4.765483,4.947005,4.872625,5.144602,5.031794,5.089739,4.942311,5.133462


In [15]:
GSE95065_merged.write_parquet(file="../data/adex-database/samples/SSc/GSE95065_merged.tsv.parquet", use_pyarrow=True)

In [4]:
# validate 
pl.read_parquet("../data/adex-database/samples/SSc/GSE95065_merged.tsv.parquet")

gene,GSM2495594,GSM2495595,GSM2495596,GSM2495597,GSM2495598,GSM2495599,GSM2495600,GSM2495601,GSM2495602,GSM2495603,GSM2495604,GSM2495605,GSM2495606,GSM2495607,GSM2495608,GSM2495609,GSM2495610,GSM2495611,GSM2495612,GSM2495613,GSM2495614,GSM2495615,GSM2495616,GSM2495617,GSM2495618,GSM2495619,GSM2495620,GSM2495621,GSM2495622,GSM2495623,GSM2495624,GSM2495625,GSM2495626
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""CEBPE""",7.430844,7.424512,7.380993,7.495515,7.424263,7.24761,7.358499,7.352183,7.373819,7.574839,7.341107,7.483675,7.384654,7.317921,7.326927,7.497311,7.343509,7.384384,7.439529,7.171636,7.541954,7.175672,7.52662,7.103011,7.094538,7.276562,7.277203,7.154872,7.350882,7.365737,7.076975,7.140006,7.11732
"""ADCYAP1R1""",5.94925,5.912904,5.952583,6.136321,5.660838,4.507445,5.995402,5.392767,6.156154,5.868257,5.963205,6.043992,5.441751,5.91522,5.651909,5.814725,6.128093,5.859722,5.756369,5.602133,5.668191,6.470656,5.523302,5.818856,5.381958,5.952577,6.267259,5.528522,6.896724,5.5735,6.109171,5.765169,5.834804
"""COL7A1""",6.533028,6.441243,6.26865,6.402277,6.406926,6.124735,6.913235,6.183706,6.143744,6.092093,6.323905,6.448042,6.687022,6.449526,6.58893,6.058564,6.496673,6.376913,6.206048,6.574422,6.464814,6.64254,6.144032,6.473642,6.567916,6.610596,6.442491,6.794229,6.435983,6.277183,6.302032,6.382896,6.893414
"""KLF6""",6.90694,6.48762,6.229516,6.134647,6.217019,6.197873,5.779039,6.464217,6.489212,6.001884,6.226221,6.309929,5.870903,5.843094,5.924252,6.168541,6.366553,6.147186,5.792084,6.085894,5.959874,5.963343,6.404039,6.674311,6.161022,6.13027,6.000803,5.985383,5.773169,5.903811,6.093964,5.95448,6.172006
"""CSF2RA""",5.482237,5.547813,6.167729,5.801479,5.7241,6.074025,5.030489,5.985279,5.512681,7.077504,6.963825,5.931991,7.247194,7.027666,7.610069,6.462201,7.18134,7.206354,6.125501,7.399473,7.054716,6.050846,7.134491,6.054555,7.198266,6.117478,5.848947,5.850248,5.67489,5.840013,6.061546,6.261048,6.09999
"""CTBP1""",7.096109,7.070467,7.344943,7.401909,7.352289,7.293874,6.940814,7.268688,7.075635,7.617213,7.378066,7.332296,7.393262,7.464938,7.756849,7.379056,7.383231,7.848183,7.146071,7.513861,7.4345,7.320879,7.648898,7.124524,7.703255,7.059668,6.957033,7.352952,7.183082,7.296272,7.286571,7.572689,7.418754
"""DIAPH1""",8.14523,8.312656,7.992397,8.272858,8.040403,7.828828,8.157008,8.061688,8.004696,7.385955,7.887343,8.037436,8.048556,7.902219,7.907266,8.116818,7.760369,7.803573,8.014856,7.750942,7.780054,8.198553,8.027947,7.939627,7.718404,8.141551,8.181171,8.108116,8.095633,7.884621,7.977461,7.938129,8.151805
"""AGER""",6.496517,6.508898,6.538657,6.435531,6.282903,5.896218,6.146781,6.536278,6.593639,6.228704,6.463922,6.752157,6.412501,6.361711,6.306303,6.793278,6.331338,6.189678,6.208045,6.186679,6.070997,6.663842,6.381444,6.595394,5.902664,6.686002,6.480337,5.997034,6.535427,6.661929,6.405839,6.227995,6.342333
"""DNASE1""",4.872944,4.847915,5.046468,5.162639,5.052784,4.968124,4.846196,4.997902,4.970702,6.169151,6.330242,4.943527,6.44802,6.408799,6.298719,5.029668,6.074697,6.764309,4.955496,6.080928,6.642391,4.991812,6.655077,4.973225,6.002208,4.765483,4.947005,4.872625,5.144602,5.031794,5.089739,4.942311,5.133462


### GSE55098

The second file seems to only have 5 samples. Easier to just discard it. This whole dataset is too small anyway. 

In [3]:
GSE55098_one = pl.read_parquet("../data/adex-database/samples/GSE55098.tsv.parquet")
GSE55098_two = pl.read_parquet("../data/adex-database/samples/GSE55098_2.tsv.parquet")

In [4]:
GSE55098_one

gene,GSM1329616,GSM1329617,GSM1329618,GSM1329619,GSM1329620,GSM1329621,GSM1329622,GSM1329623,GSM1329624,GSM1329625,GSM1329626,GSM1329627,GSM1329628,GSM1329629,GSM1329630,GSM1329631,GSM1329632,GSM1329633,GSM1329634,GSM1329635,GSM1329636,GSM1329637
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""RFC2""",7.29312,7.478815,7.163901,7.19965,7.201116,7.000996,7.605478,7.084733,7.274616,7.623437,7.695536,7.628405,7.628396,7.376713,7.502768,7.783859,7.16911,7.128845,7.331175,7.392728,7.518944,7.957457
"""HSPA6""",8.646147,9.134429,7.398783,7.617116,8.546272,8.837147,7.904668,9.397619,8.595635,9.133088,9.955381,8.704306,9.249976,8.592631,8.990663,9.506727,8.826066,10.290093,8.566601,10.476805,8.968847,9.544259
"""PAX8""",4.955674,4.633809,4.779538,4.769782,5.189039,4.969808,5.008537,5.134186,5.157049,4.728086,5.346386,4.71582,4.804079,5.088407,5.040982,4.851637,4.964125,5.303836,4.845228,4.768793,4.982103,5.013744
"""GUCA1A""",4.566657,4.454742,4.474081,4.542757,4.448803,4.056051,4.630339,4.546007,4.39032,4.147015,4.410766,4.453085,4.434004,4.438081,4.488301,4.478591,4.473089,4.383461,4.466288,4.467433,4.790837,4.408654
"""THRA""",5.744708,5.691902,5.289194,5.564487,5.790481,5.947639,5.609385,5.785431,5.950154,5.576511,5.39011,6.25262,5.490247,5.58748,5.606161,5.543975,5.974371,5.8293,6.018458,5.316998,5.420747,5.404102
"""PTPN21""",3.814043,3.634538,3.75437,4.113093,3.846746,3.789836,3.69543,3.964155,3.947626,3.931912,4.303271,4.125382,4.002965,4.243301,4.006246,4.026725,3.859124,4.029854,3.833951,3.973107,3.881679,3.737949
"""CCL5""",13.212147,13.164316,12.342226,12.964645,13.270208,13.030897,13.628946,13.347582,12.718301,13.200598,11.686295,12.914284,12.833934,12.759561,12.773942,12.006663,12.629261,12.564126,12.049704,12.413792,12.310641,12.695485
"""CYP2E1""",4.264835,4.350058,4.279817,4.497782,4.224286,4.247999,4.378694,4.466011,4.180108,4.334538,4.267884,4.295343,4.096116,4.578787,4.088221,4.24095,4.37154,4.109611,4.11845,4.183501,4.150002,4.532212
"""EPHB3""",5.521879,5.418001,5.315546,5.550489,5.666342,5.401517,5.435024,5.579085,5.631932,5.334814,6.090561,5.428605,5.513184,5.924601,5.569855,5.799532,5.658137,6.12622,5.126061,5.530971,5.530454,5.541074


In [5]:
GSE55098_two

gene,GSM1329617,GSM1329622,GSM1329627,GSM1329632,GSM1329637
str,f64,f64,f64,f64,f64
"""RFC2""",7.737012,7.836727,7.794087,7.432892,8.124732
"""HSPA6""",9.286261,8.037874,8.922395,8.983894,9.658149
"""PAX8""",4.728237,5.001011,4.880428,4.856147,4.956531
"""GUCA1A""",4.445629,4.519743,4.519743,4.524788,4.519743
"""THRA""",5.96914,5.784274,6.317857,6.025319,5.573626
"""PTPN21""",3.674374,3.846698,3.886408,3.886408,3.886408
"""CCL5""",13.148091,13.637502,12.916867,12.593212,12.699659
"""CYP2E1""",4.549542,4.557339,4.464209,4.559979,4.650939
"""EPHB3""",5.426595,5.335683,5.426595,5.499359,5.465223
